In [1]:
#!/usr/bin/env python
#SBATCH --job-name=PSNN
#SBATCH --error=%x.%j.err
#SBATCH --output=%x.%j.out
#SBATCH --mail-user=hzhao@teco.edu
#SBATCH --export=ALL
#SBATCH --time=48:00:00
#SBATCH --partition=sdil
#SBATCH --gres=gpu:1

import os
import sys
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), 'utils'))
from configuration import *
import torch
import pprint
from utils import *
import BaselineModelsILNC as B

In [2]:
args = parser.parse_args([])
args.task = 'temporized'
args.DEVICE = 'cpu'
args.SEED = 0

args.metric = 'acc_snn'
args.lnc = False

In [3]:
args = FormulateArgs(args)
print(f'Training network on device: {args.DEVICE}.')
MakeFolder(args)

Training network on device: cpu.


In [4]:
train_loader, datainfo = GetDataLoader(args, 'train')
valid_loader, datainfo = GetDataLoader(args, 'valid')
test_loader, datainfo = GetDataLoader(args, 'test')
pprint.pprint(datainfo)

{'N_class': 2,
 'N_feature': 6,
 'N_test': 25,
 'N_time': 32,
 'N_train': 70,
 'N_valid': 23,
 'dataname': 'acuteinflammation'}


In [15]:
SetSeed(args.SEED)
setup = f"baseline_model_SNN_data_{datainfo['dataname']}_seed_{args.SEED:02d}.model222"
print(f'Training setup: {setup}.')

Training setup: baseline_model_SNN_data_acuteinflammation_seed_00.model222.


In [16]:
msglogger = GetMessageLogger(args, setup)
msglogger.info(f'Training network on device: {args.DEVICE}.')
msglogger.info(f'Training setup: {setup}.')
msglogger.info(datainfo)

In [17]:
for x, y in train_loader:
    pass

In [18]:
topology = [datainfo['N_feature']] + args.hidden + [datainfo['N_class']]
msglogger.info(f'Topology of the network: {topology}.')

mlp = B.mlp(args, topology).to(args.DEVICE)
lstm = B.lstm(args, datainfo['N_feature'], datainfo['N_class']).to(args.DEVICE)
pnn = B.pNN(topology, args).to(args.DEVICE)
snn = B.SpikingNeuralNetwork(args, topology).to(args.DEVICE)

In [19]:
optimizer = torch.optim.Adam(snn.GetParam(), lr=args.LR)

In [20]:
for g in optimizer.param_groups:
    print(len(g['params']))

2


In [21]:
for n, p in snn.named_parameters():
    print(n)

model.MAC0.weight
model.SNNLayer0.SNNList.0.beta
model.SNNLayer0.SNNList.0.threshold
model.SNNLayer0.SNNList.1.beta
model.SNNLayer0.SNNList.1.threshold
model.SNNLayer0.SNNList.2.beta
model.SNNLayer0.SNNList.2.threshold
model.MAC1.weight
model.SNNLayer1.SNNList.0.beta
model.SNNLayer1.SNNList.0.threshold
model.SNNLayer1.SNNList.1.beta
model.SNNLayer1.SNNList.1.threshold


In [23]:
if os.path.isfile(f'{args.savepath}/{setup}'):
    print(f'{setup} exists, skip this training.')
    msglogger.info('Training was already finished.')
else:
    topology = [datainfo['N_feature']] + args.hidden + [datainfo['N_class']]
    msglogger.info(f'Topology of the network: {topology}.')

    snn = B.SpikingNeuralNetwork(args, topology).to(args.DEVICE)
    
    lossfunction = B.SNNLoss().to(args.DEVICE)
    optimizer = torch.optim.Adam(snn.GetParam(), lr=args.LR)

    if args.PROGRESSIVE:
        snn, best = train_pnn_progressive(snn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)
    else:
        snn, best = train_pnn(snn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)

    if best:
        if not os.path.exists(f'{args.savepath}/'):
            os.makedirs(f'{args.savepath}/')
        torch.save(snn, f'{args.savepath}/{setup}')
        msglogger.info('Training if finished.')
    else:
        msglogger.warning('Time out, further training is necessary.')

Restart previous training from 248 epoch with lr: 0.05.
2
| Epoch:    248 | Train loss: 4.5936e-01 | Valid loss: 3.8564e-01 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   1 | lr: 0.05 | Epoch time: 0.2 | Power: 0.00e+00 |
2
| Epoch:    249 | Train loss: 4.6556e-01 | Valid loss: 3.5263e-01 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   2 | lr: 0.05 | Epoch time: 0.1 | Power: 0.00e+00 |
2
| Epoch:    250 | Train loss: 4.4533e-01 | Valid loss: 3.8009e-01 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   3 | lr: 0.05 | Epoch time: 0.1 | Power: 0.00e+00 |
2
| Epoch:    251 | Train loss: 4.4413e-01 | Valid loss: 3.4499e-01 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   4 | lr: 0.05 | Epoch time: 0.1 | Power: 0.00e+00 |
2
| Epoch:    252 | Train loss: 4.4388e-01 | Valid loss: 3.5647e-01 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   5 | lr: 0.05 | Epoch time: 0.1 | Power: 0.00e+00 |
2
| Epoch:    253 | Train loss: 4.4356e-01 | Valid loss: 3.6426e-01 | Train acc


KeyboardInterrupt



TypeError: 'Namespace' object is not iterable